## Problem Description
***
Create a schedule for each 4 hour time period in a full day (6 time periods) where each full time employee works an entir 8 hour shift. Full time employees that start in a 4 hour shift window will also work the next 4 hour window. Part time employees only work a 4 hour shift.

Full time employees are paid $7.5/hour for 8 hours/day  
Part time employees are paid $9/hour for 4 hours/day

| Time Period | Min Number of Employees |
|-------------|-------------------------|
| 12am to 4am | 90                      |
|  4am to 8am | 215                     |
|  8am to 12pm| 250                     |
| 12pm to 4pm | 165                     |
|  4pm to 8pm | 300                     |
|  8pm to 12am| 125                     |


In [26]:
import numpy as np 
import pandas as pd
import pulp
import json

In [27]:
#Variables
fulltime_pay = 7.5
parttime_pay = 9.
min_employees = [90, 215, 250, 165, 300, 125]

shifts = ['12am to 4am', '4am to 8am', '8am to 12pm', '12pm to 4pm', '4pm to 8pm', '8pm to 12am']
pay_types = ['FT', 'PT']

n_shifts = len(shifts)
n_pay_types = len(pay_types)

#create a shifted index to add full time employees from previous shift
shifted_index = [i for i in range(6)]
shifted_index = shifted_index[-1:] + shifted_index[:-1]

In [30]:
model = pulp.LpProblem(name='shift_planning', sense=pulp.LpMinimize)

schedule_names = [str(i)+str(j) for i in range(1, n_shifts+1) for j in range(1, n_pay_types+1)]


schedule_variables = pulp.LpVariable.matrix('P', schedule_names,
                                             lowBound=0, cat=pulp.LpInteger)

schedule = np.array(schedule_variables).reshape(n_shifts, n_pay_types)

#The sum of each shift is greater than or equal to the shift total
for index, shift in enumerate(shifts):
    model.addConstraint(pulp.LpConstraint(
        e=schedule[index,0] + schedule[index,1] + schedule[shifted_index[index],1] ,
        sense=pulp.LpConstraintGE,
        name='min_employee_' + shift,
        rhs=min_employees[index]))

objective = pulp.lpSum(schedule[:,0]*parttime_pay*4 + schedule[:,1]*fulltime_pay*8)

model.setObjective(objective)

model.solve()

if model.status == 1:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
    print(f'objective: ${model.objective.value():,.0f}')
    output = []
    for i,var in enumerate(model.variables()):
        output.append(var.value())

    print(np.array(output).reshape(n_shifts, n_pay_types))
else:
    print(f'status: {model.status}, {pulp.LpStatus[model.status]}')
 

status: 1, Optimal
objective: $35,160
[[  0.  90.]
 [  0. 125.]
 [  0. 125.]
 [  0.  40.]
 [135. 125.]
 [  0.   0.]]


In [31]:
model

shift_planning:
MINIMIZE
36.0*P_11 + 60.0*P_12 + 36.0*P_21 + 60.0*P_22 + 36.0*P_31 + 60.0*P_32 + 36.0*P_41 + 60.0*P_42 + 36.0*P_51 + 60.0*P_52 + 36.0*P_61 + 60.0*P_62 + 0.0
SUBJECT TO
min_employee_12am_to_4am: P_11 + P_12 + P_62 >= 90

min_employee_4am_to_8am: P_12 + P_21 + P_22 >= 215

min_employee_8am_to_12pm: P_22 + P_31 + P_32 >= 250

min_employee_12pm_to_4pm: P_32 + P_41 + P_42 >= 165

min_employee_4pm_to_8pm: P_42 + P_51 + P_52 >= 300

min_employee_8pm_to_12am: P_52 + P_61 + P_62 >= 125

VARIABLES
0 <= P_11 Integer
0 <= P_12 Integer
0 <= P_21 Integer
0 <= P_22 Integer
0 <= P_31 Integer
0 <= P_32 Integer
0 <= P_41 Integer
0 <= P_42 Integer
0 <= P_51 Integer
0 <= P_52 Integer
0 <= P_61 Integer
0 <= P_62 Integer